In [ ]:
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
transactions = fs.get_feature_group("transactions", version=1)
profiles = fs.get_feature_group("profiles", version=1)
profiles_activity_10m = fs.get_feature_group("profiles_activity_10m", version=1)

In [ ]:
query = transactions.select(['fraud_label', 'amount', 'category', 'is_transaction_abroad']).join(
    profiles.select_all(include_primary_key=False, include_event_time=False), on="cc_num"
).join(
    profiles_activity_10m.select(['num_trans_per_10m', 'avg_amt_per_10m']), on="cc_num"
)

In [ ]:
fv = fs.get_or_create_feature_view(
    name="fraud_model",
    version=1,
    query=query,
    labels=['fraud_label'],
    transformation_functions={
        'country': fs.get_transformation_function(name="label_encoder"),
        'category': fs.get_transformation_function(name="label_encoder"),
    }
)

In [ ]:
query.to_string()

In [ ]:
fv.create_train_test_split(
    test_size=0.2,
    data_format="csv",
    coalesce=True,
    statistics_config={'histograms': True, 'correlations': True},
)